In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# read data

In [ ]:
#### string gene_id map
local_stringdb = os.path.join('/itf-fi-ml/shared/users/ziyuzh/svm/data/stringdb','2023')

ppidf = pd.read_csv(os.path.join(local_stringdb,'9606.protein.info.v12.0.txt'), sep='\t', header=0, usecols=['#string_protein_id', 'preferred_name'])
ppidf['preferred_name'] = ppidf['preferred_name'].str.upper()
stringId2name = ppidf.set_index('#string_protein_id')['preferred_name'].to_dict()
name2stringId = ppidf.set_index('preferred_name')['#string_protein_id'].to_dict()
ppidf = pd.read_csv(os.path.join(local_stringdb,'9606.protein.aliases.v12.0.txt'), sep='\t', header=0, usecols=['#string_protein_id', 'alias']).drop_duplicates(['alias'], keep='first')
ppidf['alias'] = ppidf['alias'].str.upper()
aliases2stringId = ppidf.set_index('alias')['#string_protein_id'].to_dict()

In [2]:
ppi_features = pd.read_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/ppi_full_400_emb.csv')
# ppi_features = pd.read_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/pre_processed_features/seq_emb/human_uniport_seqemb.csv')
# ppi_features = pd.read_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/pre_processed_features/expression_emb/exp_emb.csv')


all_df = pd.read_csv('/itf-fi-ml/shared/users/ziyuzh/svm/data/disgent_2020/disgenet_string.csv')

In [3]:
ppi_features.columns

Index(['string_id', 'feature_0', 'feature_1', 'feature_2', 'feature_3',
       'feature_4', 'feature_5', 'feature_6', 'feature_7', 'feature_8',
       ...
       'feature_190', 'feature_191', 'feature_192', 'feature_193',
       'feature_194', 'feature_195', 'feature_196', 'feature_197',
       'feature_198', 'feature_199'],
      dtype='object', length=201)

In [6]:
ppi_features

,string_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_190,feature_191,feature_192,feature_193,feature_194,feature_195,feature_196,feature_197,feature_198,feature_199
0,9606.ENSP00000483781,0.148815,-0.100394,0.139844,-0.156749,-0.000815,0.115267,0.101778,0.158580,0.287842,...,0.227863,0.086968,0.062115,0.314489,-0.148824,-0.121338,-0.245106,0.247893,-0.034497,0.038109
1,9606.ENSP00000383042,-0.037154,0.038770,-0.100821,0.165520,0.324188,0.095911,0.090766,0.336213,0.334166,...,0.209735,-0.252097,0.266917,-0.333782,-0.011449,0.064207,0.033128,0.222754,0.234885,-0.410366
2,9606.ENSP00000483903,-0.293039,-0.049834,-0.074517,-0.003607,0.067420,-0.154789,0.113428,-0.114366,0.048271,...,0.118424,0.056385,0.146475,-0.034893,0.039808,-0.165166,0.168566,-0.207136,-0.079345,0.191214
3,9606.ENSP00000254066,-0.110673,0.092843,-0.023413,0.004482,-0.127406,-0.260266,-0.047629,-0.159074,0.113255,...,-0.254870,-0.087391,0.033742,0.256390,0.146588,0.036370,-0.029299,-0.067820,0.001167,0.200372
4,9606.ENSP00000156471,0.076404,0.489907,0.282237,-0.181604,0.100491,0.077988,0.040427,0.233152,0.302573,...,-0.116222,-0.251256,0.005392,-0.005766,-0.223665,0.217651,-0.143817,0.251846,-0.050136,-0.510922
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17571,9606.ENSP00000431719,-0.292983,0.074023,-0.069144,-0.190167,0.014507,-0.475752,0.288241,0.033909,0.067345,...,-0.317928,0.015213,0.695372,0.828671,-0.122853,-0.042769,-0.114056,-0.542258,0.228939,-0.043713
17572,9606.ENSP00000247271,-0.209327,0.299719,-0.233054,-0.376629,-0.043665,0.144388,-0.019789,0.191984,0.482682,...,0.618524,0.084833,0.137700,0.045433,-0.014354,-0.461460,0.253970,-0.010454,-0.328117,-0.319897
17573,9606.ENSP00000357901,0.298814,0.042844,-0.416560,-0.097715,0.012850,0.111355,0.044981,0.104989,0.202293,...,-0.371910,-0.500772,0.363082,0.303393,0.278397,-0.425239,-0.300316,-0.502220,-0.040074,-0.113128
17574,9606.ENSP00000452662,0.030680,0.038526,-0.339246,-0.977174,0.256245,-0.386663,-0.247764,0.526390,0.182296,...,0.270962,0.129991,0.172915,0.474413,-0.319234,0.087199,0.403037,-0.507184,-0.401043,-0.598365


In [7]:
def read_data(disease):
    pos_genes_list = all_df[all_df['disease_id']==disease]['string_id']
    df = ppi_features
    df['label'] = df['string_id'].isin(pos_genes_list).astype(int)
    df.head(3)

    X = df.loc[:, df.columns.str.startswith("feature_")].to_numpy()
    y = df['label'].to_numpy()
    return df, X, y

In [8]:
df, X, y = read_data('ICD10_A23')

In [10]:
df['label'].value_counts()

label
0    17574
1        2
Name: count, dtype: int64

# svm

In [ ]:
from sklearn import svm
from sklearn import metrics
import numpy as np
from sklearn.model_selection import StratifiedKFold,ParameterGrid,GridSearchCV, train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.metrics import precision_score, recall_score, roc_curve, auc
from sklearn.metrics import recall_score, precision_score, f1_score, classification_report, confusion_matrix
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import fcluster

import matplotlib.pyplot as plt



In [ ]:
def average_rank_ratio(y_scores, y_test):
    """
    Calculate the average predicted rank of true positives.

    Parameters:
    y_scores (array-like): Decision function scores from the classifier.
    y_test (array-like): True binary labels (0 for negative, 1 for positive).

    Returns:
    float: The average rank of true positives.
    """
    
    # Convert inputs to numpy arrays for consistency
    y_scores = np.array(y_scores)
    y_test = np.array(y_test)

    # Step 1: Sort scores in descending order and assign ranks
    sorted_indices = np.argsort(-y_scores)  # Negative for descending sort
    ranks = np.empty_like(sorted_indices)
    ranks[sorted_indices] = np.arange(1, len(y_scores) + 1)  # Rank starts from 1

    # Step 2: Identify true positives
    true_positive_indices = np.where(y_test == 1)[0]

    # Step 3: Extract ranks of true positives
    true_positive_ranks = ranks[true_positive_indices]

    # Step 4: Calculate the average rank of true positives
    average_rank = np.mean(true_positive_ranks)

    rank_ratio = average_rank/y_test.shape[0]

    return round(rank_ratio,4)

# def eval(clf, X_train, y_train, X_test, y_test, print_result = False):
#     clf.fit(X_train, y_train)
#     y_pred = clf.predict(X_test)
    
#     if hasattr(clf, "decision_function"):
#         y_scores = clf.decision_function(X_test)
#     else:
#         y_scores = None  # AUROC cannot be computed without scores

#     rank_ratio = average_rank_ratio(y_scores, y_test)
    
#     if -1 in y_test:
#         y_test = np.where(y_test == -1, 0, y_test)
#     if -1 in y_pred:
#         y_pred = np.where(y_pred == -1, 0, y_pred)
        
#     # Compute AUROC if scores are available
#     if y_scores is not None:
#         try:
#             auroc = roc_auc_score(y_test, y_scores)
#         except:
#             auroc = "AUROC computation failed (possibly due to label issues)"
#     else:
#         auroc = "AUROC not available (no predict_proba or decision_function)"
#     if print_result == True:
#         # Print classification metrics
#         print(classification_report(y_test, y_pred))
        
#         print(f"""
#         Recall: {recall_score(y_test, y_pred, average="binary", pos_label=1)}
#         Precision: {precision_score(y_test, y_pred, average="binary", pos_label=1)}
#         F1-score: {f1_score(y_test, y_pred, average="binary", pos_label=1)}
#         AUROC: {auroc}
#         Rank ratio: {rank_ratio * 100:.2f}%
#         """)
        
#         print("Confusion Matrix:")
#         print(confusion_matrix(y_test, y_pred))
#     # print('check')
#     return (
#         recall_score(y_test, y_pred, average="binary", pos_label=1), 
#         precision_score(y_test, y_pred, average="binary", pos_label=1), 
#         f1_score(y_test, y_pred, average="binary", pos_label=1),
#         auroc,
#         rank_ratio
#     )


# def build_normalized_linear_kernel():
#     """
#     Returns a function that computes a normalized linear kernel (cosine similarity).
#     The scaling factor C is optional and can adjust the magnitude of the kernel.
#     """
#     def normalized_linear_kernel(X, Y):
#         # Step 1: Manually compute the linear kernel (dot product)
#         K = np.dot(X, Y.T)
        
#         # Step 2: Compute norms of X and Y for normalization
#         X_norms = np.linalg.norm(X, axis=1)
#         Y_norms = np.linalg.norm(Y, axis=1)
        
#         # Avoid division by zero
#         X_norms[X_norms == 0] = 1e-10
#         Y_norms[Y_norms == 0] = 1e-10
        
#         # Normalize the kernel (cosine similarity)
#         normalization_matrix = np.outer(X_norms, Y_norms)
#         K_normalized = K / normalization_matrix
        
#         return K_normalized
    
#     return normalized_linear_kernel



# def build_normalized_poly_kernel(coef0=1.0, degree=2):
#     """
#     Returns a function that computes a normalized polynomial kernel with dynamic parameters.
    
#     Parameters:
#     - coef0: Independent term added to the dot product (default=1.0).
#     - degree: Degree of the polynomial kernel (default=2).
#     """
#     def custom_poly_kernel(X, Y):
#         # Step 1: Compute the polynomial kernel
#         K = (np.dot(X, Y.T) + coef0) ** degree
        
#         # Step 2: Compute self-similarities (for normalization)
#         K_self_X = (np.sum(X * X, axis=1) + coef0) ** degree  # Diagonal for X
#         K_self_Y = (np.sum(Y * Y, axis=1) + coef0) ** degree  # Diagonal for Y
        
#         # Step 3: Avoid division by zero (replace zeros with a small number)
#         K_self_X[K_self_X == 0] = 1e-10
#         K_self_Y[K_self_Y == 0] = 1e-10
        
#         # Step 4: Normalize the kernel
#         normalization_matrix = np.outer(np.sqrt(K_self_X), np.sqrt(K_self_Y))
#         K_normalized = K / normalization_matrix
        
#         return K_normalized
    
#     return custom_poly_kernel

In [ ]:
def getLinkageMat(model):
    children = model.children_
    cs = np.zeros(len(children))
    N = len(model.labels_)
    for i,child in enumerate(children):
        count = 0
        for idx in child:
            count += 1 if idx < N else cs[idx - N]
        cs[i] = count
    return np.column_stack([children, model.distances_, cs])

def calculate_proportion(hierarchical_labels,y_train):
    proportion = []
    unique_clusters = np.unique(hierarchical_labels)
    # Total number of samples with label 1
    total_y1 = np.sum(y_train == 1)
    # print('total positive',total_y1)
    for c in unique_clusters:
        # Indices of samples in the current cluster
        cluster_indices = np.where(hierarchical_labels == c)[0]
        
        # Number of samples with label 1 in this cluster
        cluster_y1 = np.sum(y_train[cluster_indices] == 1)
        
        # Calculate A_i using the corrected formula
        A_i = len(cluster_indices) * (1 - (cluster_y1 / total_y1))
        proportion.append(A_i)
    return proportion

def cluster_negative_sampling(hierarchical_labels,y_train,proportion,neg_size):
    neg_cluster = []
    all_sampled_negatives = []
    unique_clusters = np.unique(hierarchical_labels)
    total_proportion = sum(proportion)
    if total_proportion == 0:
        raise ValueError("Sum of proportions is zero. Check the 'proportion' calculation.")

    for index, c in enumerate(unique_clusters):
        A_i = proportion[index]
        neg_i = int((A_i / total_proportion) * neg_size)
        cluster_indices = np.where(hierarchical_labels == c)[0]
        negative_indices = np.array(cluster_indices)[y_train[cluster_indices] == 0]
        # Check if there are enough negative samples to draw from
        if len(negative_indices) == 0:
            continue  
        elif neg_i > len(negative_indices):
            neg_i = len(negative_indices)  # Adjust to the maximum available
        # Sample negative indices without replacement
        sampled_indices = np.random.choice(negative_indices, size=neg_i, replace=False)
        # Store the sampled indices and the count
        all_sampled_negatives.extend(sampled_indices)
        neg_cluster.append(neg_i)

    return all_sampled_negatives

# def generate_pesudo_labels(negative_ratio,X_train,y_train):
#     neg_size = negative_ratio * np.sum(y_train == 1)
#     # clustering
#     model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
#     model = model.fit(X_train)
#     mat = getLinkageMat(model)
#     # get first level labels
#     n_clusters = 5
#     hierarchical_labels = fcluster(mat, n_clusters, criterion='maxclust')
#     # sample negatives
#     proportion = calculate_proportion(hierarchical_labels,y_train)
#     all_sampled_negatives = cluster_negative_sampling(hierarchical_labels,y_train,proportion,neg_size)
#     # return selected train set
#     positive_indices = np.where(y_train == 1)[0]
#     balanced_indices = np.concatenate([positive_indices, all_sampled_negatives])
#     return balanced_indices

In [ ]:
# def per_disease_val(df,X,y,methods):
#     result_df = pd.DataFrame(columns=['method',"fold","kernel", "recall", "precision", "f1-score",'auroc',"rank_ratio"])
#     kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

#     for fold, (train_idx, test_idx) in enumerate(kf.split(df, df['label'])):
#         X_train = X[df.index.get_indexer(train_idx)]
#         y_train = y[df.index.get_indexer(train_idx)]

#         X_test = X[df.index.get_indexer(test_idx)]
#         y_test = y[df.index.get_indexer(test_idx)]
#         for method in methods:
#             if method == 'ooc':
#                 X_pos_train = X_train[np.where(y_train == 1)]
#                 y_pos_train = y_train[np.where(y_train == 1)]
#                 X_all_test = np.vstack((X_train[np.where(y_train != 1)], X_test))
#                 y_all_test = np.concatenate((y_train[np.where(y_train != 1)], y_test))
#                 y_test_ooc = np.where(y_all_test == 0, -1, y_all_test)
                
#                 clf = svm.OneClassSVM(nu=0.8,kernel=build_normalized_linear_kernel())
#                 result_df.loc[len(result_df.index)] = ["ooc_svm",fold,"norm_linear", *eval(clf, X_pos_train, y_pos_train, X_all_test, y_test_ooc)]
#                 clf = svm.OneClassSVM(nu= 0.8, kernel=build_normalized_poly_kernel(degree = 3))
#                 result_df.loc[len(result_df.index)] = ["ooc_svm",fold,"norm_poly", *eval(clf, X_pos_train, y_pos_train, X_all_test, y_test_ooc)]
#                 clf = svm.OneClassSVM(nu= 0.8, kernel='rbf')
#                 result_df.loc[len(result_df.index)] = ["ooc_svm",fold,"rbf", *eval(clf, X_pos_train, y_pos_train, X_all_test, y_test_ooc)]
#             elif method == 'random_negative':
#                 negative_indices = np.where(y_train == 0)[0]
#                 positive_indices = np.where(y_train == 1)[0]
#                 # Randomly select the same number of negative samples as positive samples
#                 random_neg_indices = np.random.choice(negative_indices, size=5*len(positive_indices), replace=False)
#                 # Combine positive and selected negative samples
#                 balanced_indices = np.concatenate([positive_indices, random_neg_indices])
#                 # Extract the balanced subset
#                 X_train_balance = X_train[balanced_indices]
#                 y_train_balance = y_train[balanced_indices] 
#                 unused_neg_indices = np.setdiff1d(negative_indices, random_neg_indices)
#                 X_new_test = np.vstack((X_train[unused_neg_indices], X_test))
#                 y_new_test = np.concatenate((y_train[unused_neg_indices], y_test))

#                 clf = SVC(C=0.8,kernel=build_normalized_linear_kernel())
#                 result_df.loc[len(result_df.index)] = ["random_svm",fold,"norm_linear", *eval(clf, X_train_balance, y_train_balance, X_new_test, y_new_test)]
#                 clf = SVC(C=0.8,kernel=build_normalized_poly_kernel(degree = 3))
#                 result_df.loc[len(result_df.index)] = ["random_svm",fold,"norm_poly", *eval(clf, X_train_balance, y_train_balance, X_new_test, y_new_test)]
#                 clf = SVC(C=0.8,kernel='rbf')
#                 result_df.loc[len(result_df.index)] = ["random_svm",fold,"rbf", *eval(clf, X_train_balance, y_train_balance, X_new_test, y_new_test)]
#             elif method == 'pseudo_labeling':
#                 negative_ratio = 5
#                 balanced_indices = generate_pesudo_labels(negative_ratio,X_train,y_train)

#                 all_train_indices = np.arange(len(y_train))
#                 non_selected_indices = np.setdiff1d(all_train_indices, balanced_indices)

#                 # Step 4: Create the new test set with non-selected training data + original test data
#                 X_new_test = np.vstack((X_train[non_selected_indices], X_test))
#                 y_new_test = np.concatenate((y_train[non_selected_indices], y_test))
#                 clf = SVC(C=0.8,kernel=build_normalized_linear_kernel())
#                 result_df.loc[len(result_df.index)] = ["pseudo_svm",fold,"norm_linear", *eval(clf, X_train_balance, y_train_balance, X_new_test, y_new_test)]
#                 clf = SVC(C=0.8,kernel=build_normalized_poly_kernel(degree = 3))
#                 result_df.loc[len(result_df.index)] = ["pseudo_svm",fold,"norm_poly", *eval(clf, X_train_balance, y_train_balance, X_new_test, y_new_test)]
#                 clf = SVC(C=0.8,kernel='rbf')
#                 result_df.loc[len(result_df.index)] = ["pseudo_svm",fold,"rbf", *eval(clf, X_train_balance, y_train_balance, X_new_test, y_new_test)] 
#     return result_df

In [ ]:
from sklearn.model_selection import KFold
from rdkit.ML.Scoring.Scoring import CalcBEDROC

In [ ]:
def select_parameter(X_train,y_train):
        # Define parameter grid
    param_grid = {
        'C': [0.1, 0.5, 0.8, 1, 10],
        'kernel': ['rbf'],
        'gamma': ['scale', 'auto',0.1,1,10,100]
    }

    # Initialize SVM with probability estimates
    clf = svm.SVC(probability=True)
    # Perform Grid Search with Cross-Validation using AUC as the scoring metric
    grid_search = GridSearchCV(clf, param_grid, cv=5, scoring='roc_auc', verbose=1)
    grid_search.fit(X_train,y_train)
    return grid_search.best_params_
    # print("Best cross-validation AUC:", grid_search.best_score_)

In [ ]:
def eval(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    if hasattr(clf, "decision_function"):
        y_scores = clf.decision_function(X_test)
    else:
        y_scores = None  # AUROC cannot be computed without scores

    rank_ratio = average_rank_ratio(y_scores, y_test)
    
    if -1 in y_test:
        y_test = np.where(y_test == -1, 0, y_test)
    if -1 in y_pred:
        y_pred = np.where(y_pred == -1, 0, y_pred)
        
    ############### AUCROC
    if y_scores is not None:
        try:
            auroc = roc_auc_score(y_test, y_scores)
        except:
            auroc = "AUROC computation failed (possibly due to label issues)"
    else:
        auroc = "AUROC not available (no predict_proba or decision_function)"


    ############### BEDROC
    scores = np.column_stack((y_test, y_scores))  # Stack labels and scores as columns
    scores = scores[scores[:, 1].argsort()[::-1]]  # Sort by scores in descending order

    return (
        recall_score(y_test, y_pred, average="binary", pos_label=1), 
        precision_score(y_test, y_pred, average="binary", pos_label=1), 
        f1_score(y_test, y_pred, average="binary", pos_label=1),
        auroc,
        rank_ratio,
        CalcBEDROC(scores, col=0, alpha=160.9),
        CalcBEDROC(scores, col=0, alpha=32.2),
        CalcBEDROC(scores, col=0, alpha=16.1),
        CalcBEDROC(scores, col=0, alpha=5.3)
    )

def select_pseudo_negatives(neg_num,X_train_pos,y_train_pos,X_all_neg,y_all_neg):
    # clustering
    X_train = np.vstack((X_train_pos,X_all_neg))
    y_train = np.hstack((y_train_pos,y_all_neg))
    model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
    model = model.fit(X_train)
    mat = getLinkageMat(model)
    # get first level labels
    n_clusters = 5
    hierarchical_labels = fcluster(mat, n_clusters, criterion='maxclust')
    # sample negatives
    proportion = calculate_proportion(hierarchical_labels,y_train)
    all_sampled_negatives = cluster_negative_sampling(hierarchical_labels,y_train,proportion,neg_num)

    return X_train[all_sampled_negatives], y_train[all_sampled_negatives]

In [ ]:
from sklearn import svm

In [ ]:
def evaluate_disease(X, y, methods):
    result_df = pd.DataFrame(columns=['method',"fold","para", "recall", "precision", "f1-score",'auroc',"rank_ratio",'bedroc_1','bedroc_5','bedroc_10','bedroc_30'])
    # Initialize 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=42)

    # Get the positive class samples
    X_pos = X[y == 1]
    y_pos = y[y == 1]

    # Loop through each fold
    # X_train_pos, X_test_pos, y_train_pos, y_test_pos = train_test_split(X[y == 1], y[y == 1], test_size=0.2, random_state=42)
    for fold, (train_idx, test_idx) in enumerate(kf.split(X_pos)):
        X_train_pos, X_test_pos = X_pos[train_idx], X_pos[test_idx]
        y_train_pos, y_test_pos = y_pos[train_idx], y_pos[test_idx]

        neg_num = 5*X_train_pos.shape[0]
        if 'ooc' in methods:
            X_test = np.vstack((X_test_pos, X[y == 0]))
            y_test = np.array([1]*X_test_pos.shape[0]+[-1]*X[y == 0].shape[0])

            param_grid = {
                'kernel': ['rbf'],
                'gamma': ['scale', 'auto', 0.1, 0.01, 0.001],
                'nu': [0.5, 0.8, 0.9]
            }

            grid_search = GridSearchCV(svm.OneClassSVM(), param_grid, cv=5, scoring='neg_mean_squared_error')
            grid_search.fit(X_train_pos)
            best_svm = svm.OneClassSVM(**grid_search.best_params_)
            # best_svm = svm.OneClassSVM(nu=0.8,kernel='rbf',gamma='scale')
            result_df.loc[len(result_df.index)] = ["ooc",fold,str(grid_search.best_params_), *eval(best_svm, X_train_pos, y_train_pos, X_test, y_test)]

        if 'random_negative' in methods:

            # Randomly select 'neg_num' indices
            selected_neg_indices = np.random.choice(np.where(y == 0)[0], size=neg_num, replace=False)
            X_train_neg = X[selected_neg_indices]
            y_train_neg = y[selected_neg_indices]
            X_train = np.vstack((X_train_pos, X_train_neg))
            y_train = np.hstack((y_train_pos, y_train_neg))
            # Get the remaining samples by masking the selected indices
            remaining_neg_indices = np.setdiff1d(np.where(y == 0)[0], selected_neg_indices)
            X_test_neg = X[remaining_neg_indices]
            y_test_neg = y[remaining_neg_indices]
            X_test = np.vstack((X_test_pos, X_test_neg))
            y_test = np.hstack((y_test_pos, y_test_neg))
            
            parameters = select_parameter( X_train, y_train)
            best_svm = svm.SVC(**parameters)
            result_df.loc[len(result_df.index)] = ["random_negative",fold,str(parameters), *eval(best_svm, X_train, y_train, X_test, y_test)]

        if 'pseudo_labeling' in methods:
            X_train_neg, y_train_neg = select_pseudo_negatives(neg_num, X_train_pos, y_train_pos, X[y==0], y[y==0])
            dtype = X.dtype.descr  # Get data type of elements
            X_y0_view = X[y == 0].view(dtype=[('', X.dtype)] * X.shape[1])
            X_train_neg_view = X_train_neg.view(dtype=[('', X.dtype)] * X.shape[1])
            X_test_neg = np.setdiff1d(X_y0_view, X_train_neg_view).view(X.dtype).reshape(-1, X.shape[1])  

            X_train = np.vstack((X_train_pos, X_train_neg))
            y_train = np.hstack((y_train_pos, y_train_neg))

            X_test = np.vstack((X_test_pos, X_test_neg))
            y_test = np.array([1]*X_test_pos.shape[0]+[0]*X_test_neg.shape[0])
            # y_test = np.concatenate((np.ones(X_test_pos.shape[0], dtype=int),np.zeros(X_test_neg.shape[0], dtype=int)))

            parameters = select_parameter(X_train, y_train)
            best_svm = svm.SVC(**parameters)
            result_df.loc[len(result_df.index)] = ["pseudo_labeling",fold,str(parameters), *eval(best_svm, X_train, y_train, X_test, y_test)]
    return result_df

In [ ]:
selected_diseases = (
    all_df.groupby('disease_id')
    .filter(lambda x: len(x) > 15)
    ['disease_id']
    .unique()
    .tolist()
)

In [ ]:
selected_diseases = (
    all_df.groupby('disease_id')
    .filter(lambda x: len(x) > 15)
    ['disease_id']
    .unique()
    .tolist()
)[7:]

# selected_diseases = ['ICD10_G20']
methods = ['ooc','random_negative','pseudo_labeling']
all_results = []

for disease in selected_diseases:
    methods = ['ooc', 'random_negative', 'pseudo_labeling']
    df, X, y = read_data(disease)
    result_df = evaluate_disease(X, y, methods)
    result_df.to_csv('/itf-fi-ml/shared/users/ziyuzh/svm/results/per_disease/'+disease+'.csv',index = False)
    # Calculate mean metrics
    mean_df = result_df.groupby(['method'])[["recall", "precision", "f1-score",'auroc',"rank_ratio",'bedroc_1','bedroc_5','bedroc_10','bedroc_30']].mean().reset_index()

    # Add disease information
    mean_df['disease'] = disease

    # Append to all_results list
    all_results.append(mean_df)

# Concatenate all results into a single DataFrame
final_result = pd.concat(all_results, ignore_index=True)
final_result.to_csv('/itf-fi-ml/shared/users/ziyuzh/svm/results/all_disease.csv',index=False)

In [ ]:
import os

In [ ]:
all_results = []
for disease in selected_diseases:
    result_df = pd.read_csv('/itf-fi-ml/shared/users/ziyuzh/svm/results/per_disease/'+disease+'.csv')
    # Calculate mean metrics
    mean_df = result_df.groupby(['method'])[["recall", "precision", "f1-score",'auroc',"rank_ratio",'bedroc_1','bedroc_5','bedroc_10','bedroc_30']].mean().reset_index()

    # Add disease information
    mean_df['disease'] = disease

    # Append to all_results list
    all_results.append(mean_df)

# Concatenate all results into a single DataFrame
final_result = pd.concat(all_results, ignore_index=True)
final_result.to_csv('/itf-fi-ml/shared/users/ziyuzh/svm/results/all_disease.csv',index=False)

In [ ]:
final_result

In [ ]:
# selected_diseases = (
#     all_df.groupby('disease_id')
#     .filter(lambda x: len(x) > 15)
#     ['disease_id']
#     .unique()
#     .tolist()
# )
# # selected_diseases = ['ICD10_G20', 'ICD10_C50']
# # selected_diseases = ['ICD10_G20']
# all_results = []

# for disease in selected_diseases:
#     methods = ['ooc', 'random_negative', 'pseudo_labeling']
#     df, X, y = read_data(disease)
#     result_df = per_disease_val(df, X, y, methods)

#     # Calculate mean metrics
#     mean_df = result_df.groupby(['method', 'kernel'])[['recall', 'precision', 
#                                                        'f1-score', 'auroc', 'rank_ratio']].mean().reset_index()
    
#     # Keep the row with the lowest rank_ratio for each method
#     mean_df = mean_df.loc[mean_df.groupby('method')['rank_ratio'].idxmin()].reset_index(drop=True)

#     # Add disease information
#     mean_df['disease'] = disease

#     # Append to all_results list
#     all_results.append(mean_df)

# # Concatenate all results into a single DataFrame
# final_result = pd.concat(all_results, ignore_index=True)

In [ ]:
final_result